In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler
from sklearn.model_selection import KFold
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,InputLayer
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import time
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from keras.utils import to_categorical
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from minisom import MiniSom

In [3]:
# reading dataset
df=pd.read_csv("D:\MachineLearningCSV\Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,389,113095465,48,24,9668,10012,403,0,201.416667,203.548293,...,32,203985.500,5.758373e+05,1629110,379,13800000.0,4.277541e+06,16500000,6737603,BENIGN
1,389,113473706,68,40,11364,12718,403,0,167.117647,171.919413,...,32,178326.875,5.034269e+05,1424245,325,13800000.0,4.229413e+06,16500000,6945512,BENIGN
2,0,119945515,150,0,0,0,0,0,0.000000,0.000000,...,0,6909777.333,1.170000e+07,20400000,6,24400000.0,2.430000e+07,60100000,5702188,BENIGN
3,443,60261928,9,7,2330,4221,1093,0,258.888889,409.702161,...,20,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN
4,53,269,2,2,102,322,51,51,51.000000,0.000000,...,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170361,55641,49,1,3,6,18,6,6,6.000000,0.000000,...,20,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN
170362,45337,217,2,1,31,6,31,0,15.500000,21.920310,...,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN
170363,22,1387547,41,46,2728,6634,456,0,66.536585,110.129945,...,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN
170364,22,207,1,1,0,0,0,0,0.000000,0.000000,...,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN


In [4]:
df.head(5)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,389,113095465,48,24,9668,10012,403,0,201.416667,203.548293,...,32,203985.500,5.758373e+05,1629110,379,13800000.0,4.277541e+06,16500000,6737603,BENIGN
1,389,113473706,68,40,11364,12718,403,0,167.117647,171.919413,...,32,178326.875,5.034269e+05,1424245,325,13800000.0,4.229413e+06,16500000,6945512,BENIGN
2,0,119945515,150,0,0,0,0,0,0.000000,0.000000,...,0,6909777.333,1.170000e+07,20400000,6,24400000.0,2.430000e+07,60100000,5702188,BENIGN
3,443,60261928,9,7,2330,4221,1093,0,258.888889,409.702161,...,20,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN
4,53,269,2,2,102,322,51,51,51.000000,0.000000,...,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN


In [5]:
df.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,170366.000000,1.703660e+05,170366.000000,170366.000000,1.703660e+05,1.703660e+05,170366.000000,170366.000000,170366.000000,170366.000000,...,170366.000000,170366.000000,1.703660e+05,1.703660e+05,1.703660e+05,1.703660e+05,1.703660e+05,1.703660e+05,1.703660e+05,1.703660e+05
mean,7897.090599,1.246354e+07,15.124620,18.022276,5.569859e+02,3.183147e+04,167.775982,22.697786,48.233250,47.235628,...,11.925108,25.584506,7.433804e+04,4.676038e+04,1.646422e+05,5.012699e+04,3.690478e+06,1.310724e+05,3.784764e+06,3.543232e+06
std,18235.489428,3.193852e+07,1123.107756,1494.492871,7.710431e+03,3.460816e+06,461.299214,37.990783,94.947821,141.927429,...,1077.898793,6.354041,6.182048e+05,3.689605e+05,9.958673e+05,5.606797e+05,1.298830e+07,1.733767e+06,1.325135e+07,1.284183e+07
min,0.000000,-1.000000e+00,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,53.000000,1.920000e+02,1.000000,1.000000,3.100000e+01,6.000000e+00,23.000000,0.000000,7.882314,0.000000,...,0.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,80.000000,3.141200e+04,2.000000,2.000000,6.800000e+01,1.340000e+02,41.000000,6.000000,38.000000,0.000000,...,1.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,443.000000,8.169818e+05,4.000000,2.000000,1.480000e+02,3.280000e+02,60.000000,41.000000,50.000000,17.897858,...,3.000000,32.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,65529.000000,1.200000e+08,200755.000000,270686.000000,1.197199e+06,6.270000e+08,23360.000000,1729.000000,4183.057143,5463.493850,...,192491.000000,60.000000,1.030000e+08,6.370000e+07,1.030000e+08,1.030000e+08,1.200000e+08,7.260000e+07,1.200000e+08,1.200000e+08


In [6]:
df[' Label'].value_counts()

BENIGN                        168186
Web Attack � Brute Force        1507
Web Attack � XSS                 652
Web Attack � Sql Injection        21
Name:  Label, dtype: int64

In [7]:
# Label Encoder
le_label=LabelEncoder()
df['label_n']=le_label.fit_transform(df[' Label'])

In [8]:
df.shape

(170366, 80)

In [9]:
# dropping the categorical Label Values
df1=df.drop(labels=' Label',axis=1)

In [10]:
df1.shape

(170366, 79)

In [11]:
X=df1.drop(labels='label_n',axis='columns')

In [12]:
X.shape

(170366, 78)

In [13]:
y=df1['label_n']

In [14]:
y.shape

(170366,)

In [15]:
y.value_counts()

0    168186
1      1507
3       652
2        21
Name: label_n, dtype: int64

In [16]:
#filling empty by zeros
X.fillna(0)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,389,113095465,48,24,9668,10012,403,0,201.416667,203.548293,...,23,32,203985.500,5.758373e+05,1629110,379,13800000.0,4.277541e+06,16500000,6737603
1,389,113473706,68,40,11364,12718,403,0,167.117647,171.919413,...,39,32,178326.875,5.034269e+05,1424245,325,13800000.0,4.229413e+06,16500000,6945512
2,0,119945515,150,0,0,0,0,0,0.000000,0.000000,...,0,0,6909777.333,1.170000e+07,20400000,6,24400000.0,2.430000e+07,60100000,5702188
3,443,60261928,9,7,2330,4221,1093,0,258.888889,409.702161,...,8,20,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0
4,53,269,2,2,102,322,51,51,51.000000,0.000000,...,1,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170361,55641,49,1,3,6,18,6,6,6.000000,0.000000,...,0,20,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0
170362,45337,217,2,1,31,6,31,0,15.500000,21.920310,...,0,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0
170363,22,1387547,41,46,2728,6634,456,0,66.536585,110.129945,...,24,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0
170364,22,207,1,1,0,0,0,0,0.000000,0.000000,...,0,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0


In [17]:
# replacing infinity value by zero
X.replace([np.inf,-np.inf],0,inplace=True)

In [18]:
X

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,389,113095465,48,24,9668,10012,403,0,201.416667,203.548293,...,23,32,203985.500,5.758373e+05,1629110,379,13800000.0,4.277541e+06,16500000,6737603
1,389,113473706,68,40,11364,12718,403,0,167.117647,171.919413,...,39,32,178326.875,5.034269e+05,1424245,325,13800000.0,4.229413e+06,16500000,6945512
2,0,119945515,150,0,0,0,0,0,0.000000,0.000000,...,0,0,6909777.333,1.170000e+07,20400000,6,24400000.0,2.430000e+07,60100000,5702188
3,443,60261928,9,7,2330,4221,1093,0,258.888889,409.702161,...,8,20,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0
4,53,269,2,2,102,322,51,51,51.000000,0.000000,...,1,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170361,55641,49,1,3,6,18,6,6,6.000000,0.000000,...,0,20,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0
170362,45337,217,2,1,31,6,31,0,15.500000,21.920310,...,0,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0
170363,22,1387547,41,46,2728,6634,456,0,66.536585,110.129945,...,24,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0
170364,22,207,1,1,0,0,0,0,0.000000,0.000000,...,0,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0


In [19]:
#checking NaN
X.isna().any().value_counts()

False    77
True      1
dtype: int64

In [20]:
# replacing NaN values by zero
X= X.replace(np.nan, 0)

In [21]:
#again checking for NaN values
X.isna().any().value_counts()

False    78
dtype: int64

In [22]:
# minmax scaler
scaler=MinMaxScaler(feature_range=(-3,3))
Data_Scaled=scaler.fit_transform(X)

In [23]:
Data_Scaled

array([[-2.96438218,  2.65477358, -2.9985953 , ..., -2.64648421,
        -2.175     , -2.66311985],
       [-2.96438218,  2.67368563, -2.99799755, ..., -2.65046173,
        -2.175     , -2.6527244 ],
       [-3.        ,  2.9972761 , -2.99554679, ..., -0.99173554,
         0.005     , -2.7148906 ],
       ...,
       [-2.99798562, -2.9306226 , -2.99880451, ..., -3.        ,
        -3.        , -3.        ],
       [-2.99798562, -2.9999896 , -3.        , ..., -3.        ,
        -3.        , -3.        ],
       [ 2.50711899, -2.99999745, -3.        , ..., -3.        ,
        -3.        , -3.        ]])

In [24]:
df=pd.DataFrame(Data_Scaled)

In [25]:
df.shape

(170366, 78)

In [26]:
df

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
0,-2.964382,2.654774,-2.998595,-2.999468,-2.951547,-2.999904,-2.896490,-3.000000,-2.711096,-2.776464,...,-2.999283,0.2,-2.988117,-2.945761,-2.905100,-2.999978,-2.31,-2.646484,-2.175,-2.663120
1,-2.964382,2.673686,-2.997998,-2.999113,-2.943047,-2.999878,-2.896490,-3.000000,-2.760294,-2.811198,...,-2.998784,0.2,-2.989612,-2.952581,-2.917034,-2.999981,-2.31,-2.650462,-2.175,-2.652724
2,-3.000000,2.997276,-2.995547,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,...,-3.000000,-3.0,-2.597489,-1.897959,-1.811650,-3.000000,-1.78,-0.991736,0.005,-2.714891
3,-2.959438,0.013097,-2.999761,-2.999845,-2.988323,-2.999960,-2.719264,-3.000000,-2.628661,-2.550066,...,-2.999751,-1.0,-3.000000,-3.000000,-3.000000,-3.000000,-3.00,-3.000000,-3.000,-3.000000
4,-2.995147,-2.999986,-2.999970,-2.999956,-2.999489,-2.999997,-2.986901,-2.823019,-2.926848,-3.000000,...,-2.999969,0.2,-3.000000,-3.000000,-3.000000,-3.000000,-3.00,-3.000000,-3.000,-3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170361,2.094630,-2.999997,-3.000000,-2.999934,-2.999970,-3.000000,-2.998459,-2.979179,-2.991394,-3.000000,...,-3.000000,-1.0,-3.000000,-3.000000,-3.000000,-3.000000,-3.00,-3.000000,-3.000,-3.000000
170362,1.151170,-2.999989,-2.999970,-2.999978,-2.999845,-3.000000,-2.992038,-3.000000,-2.977767,-2.975927,...,-3.000000,0.2,-3.000000,-3.000000,-3.000000,-3.000000,-3.00,-3.000000,-3.000,-3.000000
170363,-2.997986,-2.930623,-2.998805,-2.998980,-2.986328,-2.999937,-2.882877,-3.000000,-2.904563,-2.879055,...,-2.999252,0.2,-3.000000,-3.000000,-3.000000,-3.000000,-3.00,-3.000000,-3.000,-3.000000
170364,-2.997986,-2.999990,-3.000000,-2.999978,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,...,-3.000000,0.2,-3.000000,-3.000000,-3.000000,-3.000000,-3.00,-3.000000,-3.000,-3.000000


In [27]:
# Training the dataset using 5 fold cross validation 
kf = KFold(n_splits=5)
for train_index,test_index in kf.split(df,y):
    X_train, X_test, y_train, y_test = df.iloc[train_index], df.iloc[test_index], y.iloc[train_index], y.iloc[test_index]


In [28]:
X_train.shape

(136293, 78)

In [29]:
y_train.shape

(136293,)

In [30]:
X_test.shape

(34073, 78)

In [31]:
y_test.shape

(34073,)

In [36]:
# ANN
model= Sequential()
model.add(Dense(50,activation="relu", input_shape=(78,)))
model.add(Dense(40,activation="relu"))
model.add(Dense(30,activation="relu"))
model.add(Dense(4,activation="softmax"))


In [37]:
#compiling the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 50)                3950      
                                                                 
 dense_7 (Dense)             (None, 40)                2040      
                                                                 
 dense_8 (Dense)             (None, 30)                1230      
                                                                 
 dense_9 (Dense)             (None, 4)                 124       
                                                                 
Total params: 7,344
Trainable params: 7,344
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
#Training the model
y_test_cat = to_categorical(y_test)
start_time = time.time()
history=model.fit(X_train,y_train, epochs=100, batch_size=128)
training_time_ann = time.time() - start_time

Epoch 1/100
1065/1065 [==============================] - 5s 4ms/step - loss: 0.0297 - accuracy: 0.9889
Epoch 2/100
1065/1065 [==============================] - 5s 4ms/step - loss: 0.0284 - accuracy: 0.9894
Epoch 3/100
1065/1065 [==============================] - 5s 5ms/step - loss: 0.0280 - accuracy: 0.9895
Epoch 4/100
1065/1065 [==============================] - 5s 5ms/step - loss: 0.0277 - accuracy: 0.9894
Epoch 5/100
1065/1065 [==============================] - 5s 4ms/step - loss: 0.0273 - accuracy: 0.9898
Epoch 6/100
1065/1065 [==============================] - 5s 5ms/step - loss: 0.0275 - accuracy: 0.9897
Epoch 7/100
1065/1065 [==============================] - 5s 4ms/step - loss: 0.0270 - accuracy: 0.9900
Epoch 8/100
1065/1065 [==============================] - 5s 5ms/step - loss: 0.0270 - accuracy: 0.9898
Epoch 9/100
1065/1065 [==============================] - 5s 5ms/step - loss: 0.0266 - accuracy: 0.9901
Epoch 10/100
1065/1065 [==============================] - 5s 5ms/step - l

In [64]:
# Evaluate the model on test data
start_time = time.time()
prediction = model.predict(X_test)
y_predict = pd.DataFrame(prediction)
testing_time_ann = time.time() - start_time

1065/1065 [==============================] - 4s 3ms/step


In [69]:
# calculating evaluation metrics
prediction_cat = np.argmax(prediction, axis=1)
accuracy_ann=accuracy_score(y_test,prediction_cat)
precision_ann=precision_score(y_test,prediction_cat,average='weighted')
recall_ann=recall_score(y_test,prediction_cat,average='weighted')
f1score_ann=f1_score(y_test,prediction_cat,average='weighted')

C:\Users\ARUN\Mini\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [70]:
# printing results
print('-----------Results Of Atificial Neural Network----\n')
print("Accuracy of ANN:", accuracy_ann)
print("Precision of ANN:", precision_ann)
print("Recall of ANN:", recall_ann)
print("F1 Score of ANN:", f1score_ann)
print("Training Time of ANN:", training_time_ann)
print("Testing Time of ANN:", testing_time_ann)
print('\n--------------------End---------------------------')


-----------Results Of Atificial Neural Network----

Accuracy of ANN: 0.9965955448595663
Precision of ANN: 1.0
Recall of ANN: 0.9965955448595663
F1 Score of ANN: 0.9982948699103338
Training Time of ANN: 463.58475279808044
Testing Time of ANN: 5.427382946014404

--------------------End---------------------------


In [48]:
# Decision Tree model
decision_tree=DecisionTreeClassifier(criterion='entropy', max_depth=None, class_weight='balanced')

In [49]:
# fitting the DT model to data
start_time=time.time()
decision_tree.fit(X_train,y_train)
train_time_dt=time.time() - start_time

In [50]:
# predict the labels for test data
start_time=time.time()
prediction=decision_tree.predict(X_test)
y_predict=pd.DataFrame(prediction)
test_time_dt=time.time() - start_time

In [51]:
# calculating evaluation metrics
accuracy_dt=accuracy_score(y_test,y_predict)
precision_dt=precision_score(y_test,y_predict,average='weighted')
recall=recall_score(y_test,y_predict,average='weighted')
f1score_dt=f1_score(y_test,y_predict,average='weighted')


C:\Users\ARUN\Mini\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
# printing results
print('-----------Results Of Descion Tree----------------\n')
print("Accuracy of decision tree:", accuracy_dt)
print("Precision of decision tree:", precision_dt)
print("Recall of decision tree:", recall)
print("F1 Score of decision tree:", f1score_dt)
print("Training Time of decision tree:", train_time_dt)
print("Testing Time of decision tree:", test_time_dt)
print('\n--------------------End---------------------------')






-----------Results Of Descion Tree----------------

Accuracy of decision tree: 0.9999413024975787
Precision of decision tree: 1.0
Recall of decision tree: 0.9999413024975787
F1 Score of decision tree: 0.9999706503874148
Training Time of decision tree: 6.094947814941406
Testing Time of decision tree: 0.03267073631286621

--------------------End---------------------------


In [81]:
#K-NN model
knn=KNeighborsClassifier(n_neighbors=1)

In [82]:
# fitting the KNN model to data
start_time=time.time()
knn.fit(X_train,y_train)
train_time_knn=time.time() - start_time

In [83]:
# predict the labels for test data
start_time=time.time()
prediction=knn.predict(X_test)
y_predict=pd.DataFrame(prediction)
test_time_knn=time.time() - start_time

In [84]:
# calculating evaluation metrics
accuracy_knn=accuracy_score(y_test,y_predict)
precision_knn=precision_score(y_test,y_predict,average='weighted')
recall_knn=recall_score(y_test,y_predict,average='weighted')
f1score_knn=f1_score(y_test,y_predict,average='weighted')


C:\Users\ARUN\Mini\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [85]:
# printing results
print('-----------Results Of KNN----------------\n')
print("Accuracy of KNN:", accuracy_knn)
print("Precision of KNN:", precision_knn)
print("Recall of  KNN:", recall_knn)
print("F1 Score of KNN:", f1score_knn)
print("Training Time of KNN:", train_time_knn)
print("Testing Time of KNN:", test_time_knn)
print('\n--------------------End---------------------------')


-----------Results Of KNN----------------

Accuracy of KNN: 0.9998239074927362
Precision of KNN: 1.0
Recall of  KNN: 0.9998239074927362
F1 Score of KNN: 0.9999119459935428
Training Time of KNN: 0.6258175373077393
Testing Time of KNN: 13.309284925460815

--------------------End---------------------------


In [47]:
#Naive bayesian
naive_bayes=GaussianNB()

In [48]:
# fitting the NB model to data
start_time=time.time()
naive_bayes.fit(X_train,y_train)
train_time_nb=time.time() - start_time

In [49]:
# predict the labels for test data
start_time=time.time()
prediction=naive_bayes.predict(X_test)
y_predict=pd.DataFrame(prediction)
test_time_nb=time.time() - start_time

In [50]:
# calculating evaluation metrics
accuracy_nb=accuracy_score(y_test,y_predict)
precision_nb=precision_score(y_test,y_predict,average='weighted')
recall_nb=recall_score(y_test,y_predict,average='weighted')
f1score_nb=f1_score(y_test,y_predict,average='weighted')


C:\Users\ARUN\Mini\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [51]:
# printing results
print('-----------Results Of Naive Bayesian----------------\n')
print("Accuracy of Naive Bayesian:", accuracy_nb)
print("Precision of Naive Bayesian:", precision_nb)
print("Recall of Naive Bayesian:", recall_nb)
print("F1 Score of Naive Bayesian:", f1score_nb)
print("Training Time of Naive Bayesian:", train_time_nb)
print("Testing Time of Naive Bayesian:", test_time_nb)
print('\n--------------------End---------------------------')


-----------Results Of Naive Bayesian----------------

Accuracy of Naive Bayesian: 0.9869398057112669
Precision of Naive Bayesian: 1.0
Recall of Naive Bayesian: 0.9869398057112669
F1 Score of Naive Bayesian: 0.9934269803991079
Training Time of Naive Bayesian: 0.17039871215820312
Testing Time of Naive Bayesian: 0.13303709030151367

--------------------End---------------------------


In [52]:
# Random forest
random_forest= RandomForestClassifier(n_estimators=100, max_depth=None, class_weight='balanced')

In [53]:
# fitting the RF model to data
start_time=time.time()
random_forest.fit(X_train,y_train)
train_time_rf=time.time() - start_time


In [54]:
# predict the labels for test data
start_time=time.time()
prediction=random_forest.predict(X_test)
y_predict=pd.DataFrame(prediction)
test_time_rf=time.time() - start_time


In [55]:
# calculating evaluation metrics
accuracy_rf=accuracy_score(y_test,y_predict)
precision_rf=precision_score(y_test,y_predict,average='weighted')
recall_rf=recall_score(y_test,y_predict,average='weighted')
f1score_rf=f1_score(y_test,y_predict,average='weighted')


In [56]:
# printing results
print('-----------Results Of Random Forest----------------\n')
print("Accuracy of Random Forest:", accuracy_rf)
print("Precision of Random Forest:", precision_rf)
print("Recall of Random Forest:", recall_rf)
print("F1 Score of Random Forest:", f1score_rf)
print("Training Time of Random Forest:", train_time_rf)
print("Testing Time of Random Forest:", test_time_rf)
print('\n--------------------End---------------------------')


-----------Results Of Random Forest----------------

Accuracy of Random Forest: 1.0
Precision of Random Forest: 1.0
Recall of Random Forest: 1.0
F1 Score of Random Forest: 1.0
Training Time of Random Forest: 13.471663236618042
Testing Time of Random Forest: 0.2704634666442871

--------------------End---------------------------


In [53]:
# SVM
svm1= OneVsRestClassifier(SVC(kernel='rbf', max_iter=-1, class_weight='balanced'))

In [55]:
# fitting the SVM model to data
start_time=time.time()
svm1.fit(X_train,y_train)
train_time_svm=time.time() - start_time


In [56]:
# predict the labels for test data
start_time=time.time()
prediction=svm1.predict(X_test)
y_predict=pd.DataFrame(prediction)
test_time_svm=time.time() - start_time


In [57]:
# calculating evaluation metrics

accuracy_svm=accuracy_score(y_test,y_predict)
precision_svm=precision_score(y_test,y_predict,average='weighted')
recall_svm=recall_score(y_test,y_predict,average='weighted')
f1score_svm=f1_score(y_test,y_predict,average='weighted')


C:\Users\ARUN\Mini\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [58]:
 # printing results
print('-----------Results Of SVM----------------\n')
print("Accuracy of SVM:", accuracy_svm)
print("Precision of SVM:", precision_svm)
print("Recall of SVM:", recall_svm)
print("F1 Score of SVM:", f1score_svm)
print("Training Time of SVM:", train_time_svm)
print("Testing Time of SVM:", test_time_svm)
print('\n--------------------End---------------------------')


-----------Results Of SVM----------------

Accuracy of SVM: 0.967100049892877
Precision of SVM: 1.0
Recall of SVM: 0.967100049892877
F1 Score of SVM: 0.9832748974263334
Training Time of SVM: 1417.978317975998
Testing Time of SVM: 283.3134331703186

--------------------End---------------------------


In [71]:
# KMeans model

kmeans_clustering = KMeans(n_clusters=4, max_iter=300, random_state=42)


In [72]:
# fitting the kmeans model to data
start_time = time.time()
kmeans_clustering.fit(X_train)
training_time_km = time.time() - start_time

C:\Users\ARUN\Mini\venv\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [73]:
# predict the labels for the test data
start_time = time.time()
prediction = kmeans_clustering.predict(X_test)
y_predict =pd.DataFrame(prediction)
testing_time_km = time.time() - start_time

In [74]:
# calculating evaluation metrics
accuracy_km = accuracy_score(y_test, y_predict)
precision_km = precision_score(y_test, y_predict, average='weighted')
recall_km = recall_score(y_test, y_predict, average='weighted')
f1_score_km = f1_score(y_test, y_predict, average='weighted')

C:\Users\ARUN\Mini\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [75]:
# printing results
print('-----------Results Of Kmeans----------------\n')
print("Accuracy of Kmeans:", accuracy_km)
print("Precision of Kmeans:", precision_km)
print("Recall of Kmeans:", recall_km)
print("F1 Score of Kmeans:", f1_score_km)
print("Training Time of Kmeans:", training_time_km)
print("Testing Time of Kmeans:", testing_time_km)
print('\n--------------------End---------------------------')

-----------Results Of Kmeans----------------

Accuracy of Kmeans: 0.2952190884277874
Precision of Kmeans: 1.0
Recall of Kmeans: 0.2952190884277874
F1 Score of Kmeans: 0.45585969364633366
Training Time of Kmeans: 3.702599287033081
Testing Time of Kmeans: 0.1826643943786621

--------------------End---------------------------


In [76]:
# Expectation-Maximization model
gmm= GaussianMixture(n_components=4, max_iter=300)

In [77]:
# Train the model and measure training time
start_time = time.time()
gmm.fit(X_train)
training_time_em = time.time() - start_time

In [78]:
#predict the labels for the test data
start_time = time.time()
prediction = gmm.predict(X_test)
y_predict =pd.DataFrame(prediction)
testing_time_em = time.time() - start_time

In [79]:
# calculating evaluation metrics
accuracy_em = accuracy_score(y_test, y_predict)
precision_em = precision_score(y_test, y_predict, average='weighted')
recall_em = recall_score(y_test, y_predict, average='weighted')
f1_score_em = f1_score(y_test, y_predict, average='weighted')

C:\Users\ARUN\Mini\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [80]:
# printing results
print('-----------Results Of Expectation maximization----\n')
print("Accuracy of Expectation Maximization:", accuracy_em)
print("Precision of Expectation Maximization:", precision_em)
print("Recall of Expectation Maximization:", recall_em)
print("F1 Score of Expectation Maximization:", f1_score_em)
print("Training Time of Expectation Maximization:", training_time_em)
print("Testing Time of Expectation Maximization:", testing_time_em)
print('\n--------------------End---------------------------')

-----------Results Of Expectation maximization----

Accuracy of Expectation Maximization: 0.1880667977577554
Precision of Expectation Maximization: 1.0
Recall of Expectation Maximization: 0.1880667977577554
F1 Score of Expectation Maximization: 0.31659296954126626
Training Time of Expectation Maximization: 59.08561563491821
Testing Time of Expectation Maximization: 0.3339262008666992

--------------------End---------------------------
